In [4]:
from llm_blocks import chat_utils
from git2vec import loader
from IPython.display import display, Markdown

In [5]:
repo_url = "https://github.com/voynow/turbo-docs"
raw_repo_str = loader.load(repo_url, return_str=True)

template = """
Given the following code from {repo_url}:
{raw_repo_str}

Can you create a readme file for this repo? Requirements:
- Must be a markdown file
- Include detialed instillation and quickstart instructions
- List some ideas for contributions
"""

In [7]:
readme_chain = chat_utils.GenericChain(template=template)
resp = readme_chain(repo_url=repo_url, raw_repo_str=raw_repo_str)
display(Markdown(resp["text"]))

# Turbo Docs

Turbo Docs is a Python script that utilizes OpenAI's GPT API to generate README.md files and docstrings for your Python projects automatically.

## Installation

To use Turbo Docs, you can install it using pip:

```bash
pip install turbo_docs
```

## Quickstart

To generate a README.md or docstrings for the current directory, you can run the `generate.py` script in the `turbo_docs` folder:

```bash
# command line interface
turbo_docs [--copy] [--readme] [--docstring]
```

You can use the optional flags:
- `--copy`: Copy the directory text to clipboard.
- `--readme`: Generate README.md file.
- `--docstring`: Generate and insert docstrings for each function.

## Customization

You can modify the files and directories that are excluded from the documentation generation by editing the `exclude.toml` file in the root of the repository. 

Example syntax to exclude requirements files:

```toml
exclude = [
  "requirements.*"
]
```

## Contributions

Contributions are always welcome! Here are some ideas for contributions:

- Add support for other programming languages
- Improve the OpenAI API integration
- Add support for generating documentation for specific functions or classes
- Improve the documentation and examples

Please open an issue or submit a pull request if you have any ideas or improvements to share.

In [9]:
# example with a different model
readme_chain = chat_utils.GenericChain(template=template, model_name="gpt-4")
resp = readme_chain(repo_url=repo_url, raw_repo_str=raw_repo_str)
display(Markdown(resp["text"]))

# Turbo Docs

Turbo Docs is a Python script that utilizes OpenAI's GPT API to generate README.md files and docstrings for your Python projects automatically.

## Requirements

To use Turbo Docs, you will need to have the following Python packages installed:

- setuptools
- wheel
- twine
- requests
- openai
- click
- pyperclip
- redbaron
- gitpython
- toml

You can install them using the requirements.txt and requirements.dev.txt files in the repository.

## Installation

To install Turbo Docs, follow these steps:

1. Clone the repository:

```bash
git clone https://github.com/voynow/turbo-docs.git
```

2. Change to the `turbo-docs` directory:

```bash
cd turbo-docs
```

3. Install the required packages:

```bash
pip install -r requirements.txt
pip install -r requirements.dev.txt
```

4. Install Turbo Docs:

```bash
python setup.py install
```

## Quickstart

To generate a README.md or docstrings for the current directory, you can run the `turbo_docs` command:

```bash
turbo_docs [--copy] [--readme] [--docstring]
```

You can use the optional flags:
- `--copy`: Copy the directory text to clipboard.
- `--readme`: Generate README.md file.
- `--docstring`: Generate and insert docstrings for each function.

## Customization

You can modify the files and directories that are excluded from the documentation generation by editing the `exclude.toml` file in the root of the repository. 

Example syntax to exclude requirements files:

```toml
exclude = [
  "requirements.*"
]
```

## Modules

Turbo Docs consists of three main modules:

1. `turbo_docs.commands.docstring`: Contains functions to generate docstrings for Python functions using GPT-3 text completion model.
2. `turbo_docs.commands.readme`: Contains a function to generate a README.md file using the OpenAI API.
3. `turbo_docs.utils`: Contains utility functions and decorators for working with CLI options, directories, and the OpenAI API.

## Contributing

Contributions are always welcome! If you have ideas for improvements or bug fixes, please open an issue or submit a pull request. Some ideas for contributions include:

- Improving the quality of generated README.md files and docstrings.
- Adding support for other programming languages.
- Enhancing the customization options for excluding files and directories.
- Implementing additional command-line options for more granular control over the generation process.
- Improving error handling and user experience.
